# Fully Connected Network and Data Loading

In [ ]:
import torch
import numpy as np
import random, datetime
import matplotlib.pyplot as plt

Create here a small example set of points we want to classify.  Values with $y > \cos^2\left(\frac{1}{2}\pi x\right)$ will be considered red, otherwise blue.  We'll construct a neural network to predict blue/red values.

In [ ]:
N = 60
r = np.linspace(0, 1, N)
x, y = np.meshgrid(r, r); x = x.flatten(); y = y.flatten();

red_vals = y - np.cos(0.5*x*np.pi)**2 > 0
plt.figure(figsize=(6,6))
plt.plot(x[red_vals],y[red_vals],'r+',markersize=5,alpha=0.75)
plt.plot(x[np.logical_not(red_vals)],y[np.logical_not(red_vals)],'b+',markersize=5,alpha=0.75)
plt.plot(r, np.cos(0.5*r*np.pi)**2, 'ko-', markersize=2)

Create a nice representation of our data using PyTorch's Dataset class:

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, x, y, values):
        # Load the data here!
        x = torch.Tensor(x)
        y = torch.Tensor(y)
        self.xy = torch.column_stack((x,y)) # Combine x,y data into Nx2 tensor
        self.values = torch.Tensor(values).reshape((-1,1)) # Convert values into an Nx1 tensor
        
    def __len__(self):
        return self.xy.shape[0]
    
    def __getitem__(self, idx):
        return {
            'xy': self.xy[idx],
            'values': self.values[idx]
        }

In [ ]:
true_vals = torch.Tensor([1 if i == True else -1 for i in red_vals])
true_vals

In [ ]:
data = Dataset(x, y, true_vals)

This can be indexed like a normal array

In [ ]:
data[3]

Slicing works, too

In [ ]:
data[::300]

PyTorch has built-ins for sampling minibatches of the dataset.  This will automatically combine our sample data for us!

In [ ]:
batches = torch.utils.data.DataLoader(data, batch_size=10, shuffle=True)
next(iter(batches))

We'll go ahead and create the neural network class.  Note that we can combine layers using a `Sequential` module.

In [ ]:
# Can find a helpful list of the neural network modules at https://pytorch.org/docs/stable/nn.html

class FCN(torch.nn.Module):
    def __init__(self):
        super(FCN, self).__init__()
        N = 20
        self.fc_layers = torch.nn.Sequential(
            torch.nn.Linear(2, N), torch.nn.ReLU(),
            torch.nn.Linear(N, N), torch.nn.ReLU(),
            torch.nn.Linear(N, 1), torch.nn.Tanh()
        )

    def forward(self, xy):
        # Forward propagation
        return self.fc_layers(xy)

In [ ]:
net = FCN()
opt = torch.optim.Adam(net.parameters(), lr=0.01)
loss_history = []

The `DataLoader` class from above can be used to split the dataset into batches.

In [ ]:
num_epochs = 100
batch_size = 500
loss_fn = torch.nn.SoftMarginLoss()

for i in range(num_epochs):
    batches = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)
    for batch in batches:
        opt.zero_grad()
        predicted = net(batch['xy'])
        loss = loss_fn(predicted, batch['values'])
        loss.backward()
        opt.step()

    full_loss = loss_fn(net(data.xy), data.values)
    loss_history.append(full_loss.item())

plt.plot(loss_history)
print('Final loss value', loss_history[-1])

Lets see our predictions:

In [ ]:
plt.figure(figsize=(8,8))

pred_red = (np.array(net(data.xy).detach().numpy()) > 0).flatten()
plt.plot(x[pred_red],y[pred_red],'rs',markersize=6,alpha=0.15, label='Predicted Red')
plt.plot(x[np.logical_not(pred_red)],y[np.logical_not(pred_red)],'bs',markersize=6,alpha=0.15, label='Predicted Blue')

plt.plot(x[red_vals],y[red_vals],'r+',markersize=5, label='True Red')
plt.plot(x[np.logical_not(red_vals)],y[np.logical_not(red_vals)],'b+',markersize=5, label='True Blue')
plt.legend()